In [8]:
import tensorflow as tf
import numpy as np
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data


In [9]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:

def generator(z,reuse=None):
    with tf.variable_scope('gen',reuse=reuse):
        hidden1 = tf.layers.dense(inputs=z,units=128)
        alpha = 0.01
        hidden1 = tf.maximum(alpha * hidden1,hidden1)
        hidden2 = tf.layers.dense(inputs=hidden1,units=128)
        
        hidden2 = tf.maximum(alpha * hidden2,hidden2)
        output = tf.layers.dense(hidden2,units=784,activation=tf.nn.tanh)
        return output
    

In [11]:

def discriminator(X,reuse=None):
    with tf.variable_scope('dis',reuse=reuse):
        hidden1 = tf.layers.dense(inputs=X,units=128)
        alpha = 0.02
        hidden1 = tf.maximum(alpha*hidden1,hidden1)
        hidden2 = tf.layers.dense(inputs=hidden1,units=128)
        hidden2 = tf.maximum(alpha*hidden2,hidden2)
        logits = tf.layers.dense(hidden2,units=1)
        output = tf.sigmoid(logits)
        return output, logits
    
real_images = tf.placeholder(tf.float32,shape=[None,784])
z = tf.placeholder(tf.float32,shape=[None,100])

G = generator(z)
D_real , D_logits_real = discriminator(real_images)
D_fake, D_logits_fake = discriminator(G,reuse=True)

ValueError: Variable gen/dense/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-3-ed10ce1ecca8>", line 3, in generator
    hidden1 = tf.layers.dense(inputs=noise,units=128)
  File "<ipython-input-4-99c81b954993>", line 18, in <module>
    G = generator(z)
  File "/home/ben/anaconda3/envs/WAFcat/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/ben/anaconda3/envs/WAFcat/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3248, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/home/ben/anaconda3/envs/WAFcat/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3057, in run_cell_async
    interactivity=interactivity, compiler=compiler, result=result)


In [ ]:

def loss(logits,labels):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=labels))


In [ ]:
def main():
    
    D_real_loss = loss(D_logits_real,tf.ones_like(D_logits_real)* (0.9))
    D_fake_loss = loss(D_logits_fake,tf.zeros_like(D_logits_real))
    D_loss = D_real_loss + D_fake_loss
    G_loss = loss(D_logits_fake,tf.ones_like(D_logits_fake))
    learning_rate = 0.001
    tvars = tf.trainable_variables()
    
    d_vars = [var for var in tvars if 'dis' in var.name]
    g_vars = [var for var in tvars if 'gen' in var.name]
    
    print([v.name for v in d_vars])
    print([v.name for v in g_vars])
    
    D_trainer = tf.train.AdamOptimizer(learning_rate).minimize(D_loss, var_list=d_vars)
    G_trainer = tf.train.AdamOptimizer(learning_rate).minimize(G_loss, var_list=g_vars)
    batch_size = 100
    epochs = 500
    init = tf.global_variables_initializer()
    saver = tf.train.Saver(var_list=g_vars)
    samples = []
    with tf.Session() as sess:
    
        sess.run(init)
        
        for e in range(epochs):
            num_batches = mnist.train.num_examples // batch_size
            
            for i in range(num_batches):
                
                # Grab batch of images
                batch = mnist.train.next_batch(batch_size)
                
                # Get images, reshape and rescale to pass to D
                batch_images = batch[0].reshape((batch_size, 784))
                batch_images = batch_images*2 - 1
                
                # -1 to 1 because of tanh activation
                batch_z = np.random.uniform(-1, 1, size=(batch_size, 100))
                
                # Run optimizers
                _ = sess.run(D_trainer, feed_dict={real_images: batch_images, z: batch_z})
                _ = sess.run(G_trainer, feed_dict={z: batch_z})
            
                
            print("Currently on Epoch {} of {} total...".format(e+1, epochs))
            
            # Sample from generator as we're training for viewing afterwards
            sample_z = np.random.uniform(-1, 1, size=(1, 100))
            gen_sample = sess.run(generator(z ,reuse=True),feed_dict={z: sample_z})
            
            samples.append(gen_sample)
    